# Synthetic FX index.

This model generates a UI that allows to define the FX positions and plot the changes to the index and its constituentss over a given period in time.

In [ ]:
import refinitiv.data as rd
import ipywidgets as widgets
import pandas as pd
import numpy as np
import ipysheet
import plotly.graph_objects as go
import plotly.subplots as subplots
import plotly.figure_factory as ff
from datetime import datetime, timedelta

rd.open_session()

In [16]:
direct_qm = ['AUD=', 'BTC=', 'BWP=', 'EUR=', 'FJD=', 'FKP=', 'GBP=', 'GIP=', 'IEP=', 'MTL=',
             'NZD=', 'PGK=', 'SBD=', 'SHP=', 'TOP=', 'USD=', 'WST=', 'XAG=', 'XAU=', 'XPD=', 'XPT=']

theme = 'plotly_dark'
date = '2023-06-01'
fx = ['AUD', 'GBP', 'KRW', 'CAD', 'EUR', 'NZD', 'NOK', 'CHF']

usd_annual_exposure = [10000000.00,
                       12000000.00,
                       50000000.00,
                       20000000.00,
                       50000000.00,
                       10000000.00,
                       10000000.00,
                       10000000.00]

heat_cht = go.FigureWidget()
lin_cht = go.FigureWidget()


def fx_convert(fx):
    if fx + '=' not in direct_qm:
        fx = fx+'USD=R'
    else:
        fx += '='
    return fx


def add_row(x):
    input_table.rows += 1
    return


def del_row(x):
    input_table.rows -= 1
    return


def calculate(value):
    global usd_annual_exposure, native_annual_exposure, monthly_usd_amounts, date
    global base_devisor, rics, rates_0, rates_last3m, monthly_exposure, rebased_fx_performance
    global idx_series, idx_perform, chg_sign
    global heat_cht, lin_cht, heat

    df_table = ipysheet.to_dataframe(input_table)
    fx = df_table['Currency'].values.tolist()
    df_table['USD Amount'] = df_table['USD Amount'].astype(float)
    usd_annual_exposure = df_table['USD Amount'].values.tolist()
    date = start_datepicker.value.strftime('%Y-%m-%d')
    edate = end_datepicker.value.strftime('%Y-%m-%d')
    bmk_ric = bmk_input.value

    native_annual_exposure = []
    monthly_usd_amounts = pd.DataFrame()

    base_devisor = sum(usd_annual_exposure)*100
    rics = list(map(lambda x: fx_convert(x), fx))
    rates_0 = rd.get_history(rics, 'BID', interval='daily',
                             end=str(date), start=str(pd.to_datetime(date)-timedelta(days=1)))
    rates_0 = pd.DataFrame(rates_0.iloc[-1]).transpose()

    bmk_rates_0 = rd.get_history(bmk_ric, 'TRDPRC_1', interval='daily',
                                 end=str(date), start=str(pd.to_datetime(date)-timedelta(days=1)))
    bmk_rates_0 = pd.DataFrame(bmk_rates_0.iloc[-1]).transpose()
    bmk_rates_0.rename(
        columns={'TRDPRC_1': bmk_rates_0.columns.name}, inplace=True)

    rates_last3m = rd.get_history(
        rics, 'BID', interval='monthly', end=edate, start=date)
    bmk_rates_last3m = rd.get_history(
        bmk_ric, 'TRDPRC_1', interval='monthly', end=edate, start=date)
    bmk_rates_last3m.rename(
        columns={'TRDPRC_1': bmk_rates_last3m.columns.name}, inplace=True)

    for i, r in enumerate(rics):
        if r in direct_qm:
            native_annual_exposure.append(usd_annual_exposure[i]/rates_0[r][0])
            monthly_usd_amounts[r] = native_annual_exposure[i]*rates_last3m[r]
        else:
            native_annual_exposure.append(usd_annual_exposure[i]*rates_0[r][0])
            monthly_usd_amounts[r] = native_annual_exposure[i]/rates_last3m[r]

    monthly_exposure = pd.DataFrame(native_annual_exposure)
    monthly_exposure = monthly_exposure.T
    monthly_usd_amounts['Index'] = base_devisor/monthly_usd_amounts.T.sum()
    rebased_fx_performance = pd.concat([rates_0, rates_last3m])
    rebased_fx_performance = (round(
        (rebased_fx_performance/rebased_fx_performance.shift(1)-1)*100, 2)).dropna().T
    rebased_fx_performance.columns = list(map(lambda t: t.strftime(
        '%B')[:3], rebased_fx_performance.columns.to_pydatetime()))

    rebased_fx_performance['vs {}'.format(datetime.strptime(
        date, '%Y-%m-%d').strftime('%B')[:3])] = round((rates_last3m.iloc[-1]/rates_0.iloc[0]-1)*100, 2)

    idx_series = pd.DataFrame(
        data=100.0, index=[datetime.strptime(date, '%Y-%m-%d')], columns=['Index'])
    idx_series = pd.concat(
        [idx_series, pd.DataFrame(monthly_usd_amounts['Index'])])
    idx_perform = (idx_series['Index'][-1]-100)
    idx_perform = round(idx_perform, 2)

    bmk_series = pd.concat([bmk_rates_0, bmk_rates_last3m])
    bmk_series = bmk_series / bmk_series.iloc[0] * 100

    if np.sign(idx_perform) < 0:
        chg_sign = ''
    else:
        chg_sign = ' +'

    lin_cht.data = []
    trace1 = go.Scatter(
        mode='lines',
        x=idx_series.index,
        y=idx_series['Index'],
        name='My Index'
    )

    trace2 = go.Scatter(
        mode='lines',
        x=bmk_series.index,
        y=bmk_series[bmk_ric],
        name=bmk_ric+' rebased'
    )

    lin_cht.add_trace(trace1)
    lin_cht.add_trace(trace2)
    lin_cht.layout.title = 'My Index, {}{}%'.format(chg_sign, str(idx_perform))
    lin_cht.layout.yaxis = dict(title='Index value')
    lin_cht.layout.template = theme

    heat = ff.create_annotated_heatmap(y=fx,
                                       z=rebased_fx_performance.iloc[:, 1:].values.tolist(
                                       ),
                                       x=rebased_fx_performance.iloc[:, 1:].columns.tolist(
                                       ),
                                       showscale=True
                                       )

    heat_cht.data[0].x = heat.data[0].x
    heat_cht.data[0].y = heat.data[0].y
    heat_cht.data[0].z = heat.data[0].z
    heat_cht.layout['annotations'] = heat.layout['annotations']

    heat_cht.update_layout(
        autosize=False,
        height=300,
        width=500,
        margin=dict(
            t=40, b=5, l=5, r=5
        )
    )
    heat_cht.layout.title = 'Rebased FX performance, %'
    heat_cht.layout.template = theme
    return


native_annual_exposure = []
monthly_usd_amounts = pd.DataFrame()

base_devisor = sum(usd_annual_exposure)*100
rics = list(map(lambda x: fx_convert(x), fx))
bmk_ric = '.DXY'
rates_0 = rd.get_history(rics, 'BID', interval='daily',
                         end=str(date), start=str(pd.to_datetime(date)-timedelta(days=1)))
rates_0 = pd.DataFrame(rates_0.iloc[-1]).transpose()

bmk_rates_0 = rd.get_history(bmk_ric, 'TRDPRC_1', interval='daily',
                             end=str(date), start=str(pd.to_datetime(date)-timedelta(days=1)))
bmk_rates_0 = pd.DataFrame(bmk_rates_0.iloc[-1]).transpose()
bmk_rates_0.rename(
    columns={'TRDPRC_1': bmk_rates_0.columns.name}, inplace=True)

rates_last3m = rd.get_history(
    rics, 'BID', interval='monthly', start=date, count=1000)
bmk_rates_last3m = rd.get_history(
    bmk_ric, 'TRDPRC_1', interval='monthly', start=date, count=1000)
bmk_rates_last3m.rename(
    columns={'TRDPRC_1': bmk_rates_last3m.columns.name}, inplace=True)

for i, r in enumerate(rics):
    if r in direct_qm:
        native_annual_exposure.append(usd_annual_exposure[i]/rates_0[r][0])
        monthly_usd_amounts[r] = native_annual_exposure[i]*rates_last3m[r]
    else:
        native_annual_exposure.append(usd_annual_exposure[i]*rates_0[r][0])
        monthly_usd_amounts[r] = native_annual_exposure[i]/rates_last3m[r]

monthly_exposure = pd.DataFrame(native_annual_exposure)
monthly_exposure = monthly_exposure.T
monthly_usd_amounts['Index'] = base_devisor/monthly_usd_amounts.T.sum()
rebased_fx_performance = pd.concat([rates_0, rates_last3m])
rebased_fx_performance = (round(
    (rebased_fx_performance/rebased_fx_performance.shift(1)-1)*100, 2)).dropna().T
rebased_fx_performance.columns = list(map(lambda t: t.strftime(
    '%B')[:3], rebased_fx_performance.columns.to_pydatetime()))

rebased_fx_performance['vs {}'.format(datetime.strptime(
    date, '%Y-%m-%d').strftime('%B')[:3])] = round((rates_last3m.iloc[-1]/rates_0.iloc[0]-1)*100, 2)

idx_series = pd.DataFrame(
    data=100.0, index=[datetime.strptime(date, '%Y-%m-%d')], columns=['Index'])

idx_series = pd.concat(
    [idx_series, pd.DataFrame(monthly_usd_amounts['Index'])])
idx_perform = (idx_series['Index'][-1]-100)
idx_perform = round(idx_perform, 2)

bmk_series = pd.concat([bmk_rates_0, bmk_rates_last3m])
bmk_series = bmk_series / bmk_series.iloc[0] * 100

if np.sign(idx_perform) < 0:
    chg_sign = ''
else:
    chg_sign = ' +'

trace1 = go.Scatter(
    mode='lines',
    x=idx_series.index,
    y=idx_series['Index'],
    name='My Index (inverted)'
)

trace2 = go.Scatter(
    mode='lines',
    x=bmk_series.index,
    y=bmk_series[bmk_ric],
    name=bmk_ric+' rebased'
)

lin_cht.update_layout(
    autosize=False,
    height=300,
    width=500,
    margin=dict(
        t=40, b=5, l=5, r=5
    ),
    legend=dict(
        orientation='h',
        yanchor='top',
        xanchor='center',
        y=1.05,
        x=0.1
    )

)

lin_cht.add_trace(trace1)
lin_cht.add_trace(trace2)

lin_cht.layout.title = 'My Index, {}{}%'.format(chg_sign, str(idx_perform))
lin_cht.layout.yaxis = dict(title='Index value')
lin_cht.update_layout(legend=dict(
    orientation='h', yanchor='top', xanchor='center', y=1.05, x=0.1))
lin_cht.layout.template = theme

heat = ff.create_annotated_heatmap(y=fx,
                                   z=rebased_fx_performance.iloc[:, 1:].values.tolist(
                                   ),
                                   x=rebased_fx_performance.iloc[:, 1:].columns.tolist(
                                   ),
                                   showscale=True
                                   )
heat_cht = go.FigureWidget(heat)
heat_cht.update_layout(
    autosize=False,
    height=300,
    width=500,
    margin=dict(
        t=40, b=5, l=5, r=5
    )
)
heat_cht.layout.title = 'Rebased FX performance, %'
heat_cht.layout.template = theme

bmk_input = widgets.Text(description='Benchmark:',
                         value=bmk_ric, continuous_update=False)
bmk_input.observe(calculate, names='value')

start_datepicker = widgets.DatePicker(
    description='Start date:', continuous_update=False)
start_datepicker.value = datetime.strptime(date, '%Y-%m-%d')
start_datepicker.observe(calculate, names='value')

end_datepicker = widgets.DatePicker(
    description='End date:', continuous_update=False)
end_datepicker.value = datetime.now()
end_datepicker.observe(calculate, names='value')

datepickers = widgets.HBox([bmk_input, start_datepicker, end_datepicker])
all_charts = widgets.HBox([lin_cht, heat_cht])
graphs = widgets.VBox([datepickers, all_charts])

df_input = pd.DataFrame(data=[fx, usd_annual_exposure], index=[
                        'Currency', 'USD Amount'])
input_table = ipysheet.from_dataframe(df_input.T)
input_table.row_headers = False
btn_add_row = widgets.Button(description='Add row')
btn_add_row.on_click(add_row)
btn_del_row = widgets.Button(description='Remove row')
btn_del_row.on_click(del_row)
btn_calculate = widgets.Button(description='Calculate')
btn_calculate.on_click(calculate)
header = widgets.HBox([btn_add_row, btn_del_row, btn_calculate])
editor = widgets.VBox([header, input_table])

applet = widgets.Tab()
applet.children = [graphs, editor]
applet.set_title(0, 'Charts')
applet.set_title(1, 'Editor')
applet

In [10]:
rd.close_session()